In [26]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from PIL import Image

# Memuat dataset Fashion MNIST
(train_images_mnist, train_labels_mnist), (test_images_mnist, test_labels_mnist) = fashion_mnist.load_data()

# Memuat file CSV yang berisi label dari dataset FashionImages
df = pd.read_csv('mytradataset/labels(1).csv')

# Mengambil kolom 'articleType' sebagai label FashionImages
fashionimage_labels = df.iloc[1:, 0]

# Memilih hanya 4190 data pertama dari label FashionImage
fashionimage_labels = fashionimage_labels[:4191].values

# Menggabungkan label FashionImages dengan label MNIST
merged_labels = np.concatenate([fashionimage_labels, np.zeros(len(train_labels_mnist[:4190])), np.zeros(len(test_labels_mnist[:4190]))], axis=0)

# Menggabungkan gambar dari folder 'images' dengan dataset MNIST
image_folder = 'mytradataset/images(1)/'
image_data = []
image_count = 0
for filename in os.listdir(image_folder):
    if image_count >= 4190:
        break
    image_path = os.path.join(image_folder, filename)
    image = Image.open(image_path)
    
    # Mengubah gambar menjadi skala abu-abu (grayscale)
    image = image.convert('L')
    
    # Menyesuaikan dimensi gambar menjadi 28x28
    image = image.resize((28, 28))
    
    image_data.append(np.array(image))
    image_count += 1

# Konversi data gambar menjadi array numpy
image_data_fashion = np.array(image_data)
print("Shape data gambar:", image_data_fashion.shape)
print("Shape label:", fashionimage_labels.shape)

# Memastikan jumlah gambar dan label sama
num_images = len(image_data_fashion)
train_images_mnist = train_images_mnist[:num_images]
test_images_mnist = test_images_mnist[:num_images]
merged_labels = merged_labels[:num_images]

# Menggabungkan data gambar dari FashionImage dan MNIST
image_data = np.concatenate([image_data_fashion, train_images_mnist, test_images_mnist], axis=0)

# Melihat shape data gambar dan label
print("Shape data gambar:", image_data.shape)
print("Shape label:", merged_labels.shape)


Shape data gambar: (4190, 28, 28)
Shape label: (4189,)
Shape data gambar: (12570, 28, 28)
Shape label: (4190,)


In [14]:
# Mengubah label menjadi tipe data 'str'
merged_labels = merged_labels.astype(str)

# Mengubah label menjadi one-hot encoding
label_encoder = LabelEncoder()
merged_labels_encoded = label_encoder.fit_transform(merged_labels)

# Membagi data menjadi train, validation, dan test sets
train_images, test_images, train_labels, test_labels = train_test_split(image_data, merged_labels_encoded, test_size=0.2, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.25, random_state=42)

# Normalisasi pixel intensitas gambar ke rentang 0-1
train_images = train_images / 255.0
val_images = val_images / 255.0
test_images = test_images / 255.0

# Mengubah dimensi gambar menjadi format yang sesuai untuk Conv2D
train_images = train_images.reshape(-1, 28, 28, 1)
val_images = val_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)

ValueError: Found input variables with inconsistent numbers of samples: [4190, 64190]